# Westeros Tutorial Part 2c - Flexible generation

In the other tutorials (``westeros_emissions_bounds.ipynb`` and ``westeros_emissions_taxes.ipynb``), we showed how to introduce emissions into a stylized energy system model, and what happens if you put a constraint on total CO2 emissions. We also showed how to introduce the requirement to have sufficient dispatchable (firm) capacity in tutorial ``westeros_firm_capacity.ipynb``.

In this tutorial notebook, we will illustrate how to add other considerations, in particular the requirement to have sufficient flexible generation capacity to mitigate short-term demand fluctuations.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [ ]:
import pandas as pd
import ixmp
import message_ix

from message_ix.util import make_df

%matplotlib inline

In [ ]:
mp = ixmp.Platform()

In [ ]:
base = message_ix.Scenario(mp, model='Westeros Electrified', scenario='baseline')

In [ ]:
# We clone a scenario from the baseline scenario
model = 'Westeros Electrified'
scen = base.clone(model, 'flexibile_generation',
                  'illustration of flexible-generation formulation',
                  keep_solution=False)
scen.check_out()

# we get the years of installing capacities (vintage) and years of those capacities being active
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

## Add a carbon tax

Then, we add a carbon tax to motivate the use of low-carbon technologies in the system. We do this similar to the process explained in the tutorial for adding emissions taxes (`westeros_emission_taxes.ipynb`).

In [ ]:
# first we introduce the emission specis CO2 and the emission category GHG
scen.add_set('emission', 'CO2')
scen.add_cat('emission', 'GHG', 'CO2')

# we now add CO2 emissions to the coal powerplant
base_emission_factor = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'unit': 'tCO2/kWa',
}

# adding the new unit to the model library
mp.add_unit('tCO2/kWa')

emission_factor = make_df(base_emission_factor, technology= 'coal_ppl', emission= 'CO2', value = 7.4)
scen.add_par('emission_factor', emission_factor)

In [ ]:
base_tax_emission = {
    'node': country,
    'type_year': [700,710,720],
    'type_tec': 'all',
    'unit': 'USD/tCO2',
    'type_emission': 'GHG',
    'value': [10., 20., 30.]
}

tax_emission = make_df(base_tax_emission)
scen.add_par('tax_emission', tax_emission)

## Describing flexibility requirements

Demand and supply of electricity varies over time. 
To meet the demand reliably, supply technologies must not only maintain firm capacity at any time, but also be flexible enough to ramp up and down their generation when required by the demand.
Some renewable energy technologies like wind power are "variable", meaning that their generation changes based on wind availability and not based on the electricity demand. 
As such, to balance the variability of wind power, the system requires flexibility from other units.

In [ ]:
base_flexibility_factor = pd.DataFrame({
        'node_loc': country,
        'commodity': 'electricity',
        'level' : 'secondary',
        'mode': 'standard',
        'unit': '-',
        'time': 'year',
        'year_vtg': vintage_years,
        'year_act': act_years,
})

base_rating = pd.DataFrame({
        'node': country,
        'commodity': 'electricity',
        'level' : 'secondary', 
        'unit': '-',
        'time': 'year',
        'year_act': model_horizon})

In [ ]:
# adding two different ratings for flexibility 
scen.add_set('rating', ['r1', 'r2'])

# assuming a flexibility requirment of 10% for load 
flexibility_factor = make_df(base_flexibility_factor, technology='grid', rating='unrated', value=-0.1)
scen.add_par('flexibility_factor', flexibility_factor)

# dividing wind generation into two parts (bins) for each rating (r1=20% and r2=80%) 
rating_bin = make_df(base_rating, technology='wind_ppl', value=0.2, rating='r1')
scen.add_par('rating_bin', rating_bin)
rating_bin = make_df(base_rating, technology='wind_ppl', value=0.8, rating='r2')
scen.add_par('rating_bin', rating_bin)

# assuming a flexibility requirment of 25% for wind in rating r1
flexibility_factor = make_df(base_flexibility_factor, technology='wind_ppl', rating='r1', value=-0.25)
scen.add_par('flexibility_factor', flexibility_factor)

# assuming a flexibility requirment of 45% for wind in rating r2
flexibility_factor = make_df(base_flexibility_factor, technology='wind_ppl', rating='r2', value=-0.45)
scen.add_par('flexibility_factor', flexibility_factor)

# assuming a flexibility provision of 80% for coal power plant
flexibility_factor = make_df(base_flexibility_factor, technology='coal_ppl', rating='unrated', value=0.8)
scen.add_par('flexibility_factor', flexibility_factor)

As we could see, the need for flexibility is added by negative values to the parameter `flexibility_factor`, while the provision of flexibility is specified by positive values. For more information please refer to the [mathematical specification of flexibility](https://docs.messageix.org/en/stable/model/MESSAGE/model_core.html#system-reliability-and-flexibility-requirements) in the model.

## Commit and Solve

In [ ]:
scen.commit(comment='define parameters for flexibile-generation implementation')
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var('OBJ')['lvl']

## Plotting

In [ ]:
from message_ix.reporting import Reporter
from message_ix.util.tutorial import prepare_plots

rep = Reporter.from_scenario(scen)
prepare_plots(rep)

In [ ]:
rep.set_filters(t=["coal_ppl", "wind_ppl"])
rep.get("plot activity")

In [ ]:
rep.get("plot capacity")

In [ ]:
rep.set_filters(t=None, c=["light"])
rep.get("plot prices")

## Close the connection to the database

In [ ]:
mp.close_db()

## Questions and discussion
- Run the same scenario with and without adding `flexibility_factor`. what is the difference in the results when we consider the flexibility requirements?